# Product Sales Optimization
## Goal: Help a company increase profit by identifying which products or regions are underperforming despite high discounts.

1. Business Problem:

- A national retailer is running aggressive discounts across regions. But profit margins are shrinking. Leadership wants to know:

    - Are discounts actually helping revenue?

    - Which segments or regions are hurting profitability?

    - How should we change pricing strategy?

2. Approach:

    - Call API to retrieve data and load into SQLite database

    - Use SQL to analyze order-level data, grouping by region, product category, and discount band

    - Calculate profit margin and return on discount per product segment

    - Use Power BI to visualize patterns and build a dashboard for decision-makers

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kaggle
import zipfile
import os
import sqlite3
from tabulate import tabulate

### Confirm Kaggle API connection and Change Working Directory for Dataset

In [3]:
#Confirms API connection
kaggle.api.authenticate()
working_dir = '/Users/nathanzlomke/Documents/Datasets'
os.chdir(working_dir)
os.getcwd()
os.makedirs('Kaggle', exist_ok=True)

In [4]:
# Download Superstore sales forecasting dataset
dataset = 'vivek468/superstore-dataset-final'
download_path = 'Kaggle/'

kaggle.api.dataset_download_files(dataset, path=download_path, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/vivek468/superstore-dataset-final


In [5]:
os.chdir(working_dir+'/Kaggle')

In [6]:
ls

Sample - Superstore.csv  superstore.db            train.csv
Superstore.csv           superstore_train.db


In [7]:
mv 'Sample - Superstore.csv' Superstore.csv

In [8]:
ls

Superstore.csv       superstore_train.db
superstore.db        train.csv


In [ ]:
df = pd.read_csv('Superstore.csv', index_col=0, encoding='ISO-8859-1')
df.index.name = None
df.head(2)


,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820


In [28]:
#Bins discounts into intervals
df['discount_type'] = pd.cut(df['Discount'], bins=[-.01,0,.2,.4,.6,.8,1], labels=['None','Small', 'Medium', 'Large', 'Deep','Clearance'])

#Calculate original price before discount (sales)/(discount) = original price
df['Original_Price']= np.where(df['Discount'] != 0, df['Sales']/df['Discount'], df['Sales'])

df.head(10)

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,discount_type,Original_Price
1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,None,261.96
2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,None,731.94
3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,...,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,None,14.62
4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,...,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,Large,2127.95
5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,...,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,Small,111.84
6,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,...,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1694,None,48.86
7,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,...,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.2800,4,0.00,1.9656,None,7.28
8,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,...,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.1520,6,0.20,90.7152,Small,4535.76
9,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,...,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by S...,18.5040,3,0.20,5.7825,Small,92.52
10,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,...,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.9000,5,0.00,34.4700,None,114.90


In [29]:

#Load into SQLite
sqlite_path = 'superstore.db'
connection = sqlite3.connect(sqlite_path)

df.to_sql('superstore', connection, if_exists='replace', index=False)

9994

In [30]:
cursor.execute("PRAGMA table_info(superstore)")
columns_info = cursor.fetchall()
for col in columns_info:
    print(f"Column name: {col[1]} | Type: {col[2]}")

Column name: Order ID | Type: TEXT
Column name: Order Date | Type: TEXT
Column name: Ship Date | Type: TEXT
Column name: Ship Mode | Type: TEXT
Column name: Customer ID | Type: TEXT
Column name: Customer Name | Type: TEXT
Column name: Segment | Type: TEXT
Column name: Country | Type: TEXT
Column name: City | Type: TEXT
Column name: State | Type: TEXT
Column name: Postal Code | Type: INTEGER
Column name: Region | Type: TEXT
Column name: Product ID | Type: TEXT
Column name: Category | Type: TEXT
Column name: Sub-Category | Type: TEXT
Column name: Product Name | Type: TEXT
Column name: Sales | Type: REAL
Column name: Quantity | Type: INTEGER
Column name: Discount | Type: REAL
Column name: Profit | Type: REAL
Column name: discount_type | Type: TEXT
Column name: Original_Price | Type: REAL


In [31]:
# Optional: check if it worked
cursor = connection.cursor()
cursor.execute("SELECT * FROM superstore LIMIT 5")
rows = cursor.fetchall()
column_names = [col[0] for col in cursor.description]

print(tabulate(rows, headers=column_names, tablefmt="psql"))


+----------------+--------------+-------------+----------------+---------------+-----------------+-----------+---------------+-----------------+------------+---------------+----------+-----------------+-----------------+----------------+-------------------------------------------------------------+---------+------------+------------+-----------+-----------------+------------------+
| Order ID       | Order Date   | Ship Date   | Ship Mode      | Customer ID   | Customer Name   | Segment   | Country       | City            | State      |   Postal Code | Region   | Product ID      | Category        | Sub-Category   | Product Name                                                |   Sales |   Quantity |   Discount |    Profit | discount_type   |   Original_Price |
|----------------+--------------+-------------+----------------+---------------+-----------------+-----------+---------------+-----------------+------------+---------------+----------+-----------------+-----------------+----------

Use SQL to analyze order-level data, grouping by region, product category, and discount type

In [90]:
#Groupby region

cursor.execute("""select Region, sum(Sales), sum(Quantity), sum(Original_Price), sum(Profit), (sum(Profit)/sum(Sales)*100) as 'Profit_Margin'
               FROM Superstore 
               group by Region 
               """)
rows=cursor.fetchall()
column_names = [col[0] for col in cursor.description]
print('Grouped by Region, Discount Type')
print(tabulate(rows, headers=column_names, tablefmt='psql'))


Grouped by Region, Discount Type
+----------+--------------+-----------------+-----------------------+---------------+-----------------+
| Region   |   sum(Sales) |   sum(Quantity) |   sum(Original_Price) |   sum(Profit) |   Profit_Margin |
|----------+--------------+-----------------+-----------------------+---------------+-----------------|
| Central  |       501240 |            8780 |           1.28259e+06 |       39706.4 |         7.92163 |
| East     |       678781 |           10618 |           1.90301e+06 |       91522.8 |        13.4834  |
| South    |       391722 |            6209 |      885178           |       46749.4 |        11.9343  |
| West     |       725458 |           12266 |           2.46827e+06 |      108418   |        14.9448  |
+----------+--------------+-----------------+-----------------------+---------------+-----------------+


In [104]:
#Groupby region

cursor.execute("""select sum(Quantity) as 'Total Volume',sum(Sales) as 'Total Revenue', sum(Profit) as 'Total Profit', 
               sum(Profit)/sum(Sales)*100 as 'Profit Margin', 
               sum(Original_Price-Sales) as 'Total Discounts', 
               sum(Original_Price-Sales)/sum(Original_Price) as 'Discount Loss %', 
               sum(sales)/sum(original_price)*100 as 'Realization Rate',
               sum(Profit)/sum(original_price-sales) as 'ROD Profit',
               sum(Sales)/sum(original_price-sales) as 'ROD Revenue'
               
               FROM Superstore 
               """)
rows=cursor.fetchall()
column_names = [col[0] for col in cursor.description]
print('Grouped by Region, Discount Type')
print(tabulate(rows, headers=column_names, tablefmt='psql'))


Grouped by Region, Discount Type
+----------------+-----------------+----------------+-----------------+-------------------+-------------------+--------------------+--------------+---------------+
|   Total Volume |   Total Revenue |   Total Profit |   Profit Margin |   Total Discounts |   Discount Loss % |   Realization Rate |   ROD Profit |   ROD Revenue |
|----------------+-----------------+----------------+-----------------+-------------------+-------------------+--------------------+--------------+---------------|
|          37873 |      2.2972e+06 |         286397 |         12.4672 |       4.24186e+06 |          0.648695 |            35.1305 |    0.0675169 |      0.541556 |
+----------------+-----------------+----------------+-----------------+-------------------+-------------------+--------------------+--------------+---------------+


In [105]:
#Groupby region

cursor.execute("""select 
               avg(Quantity) as 'Total Volume',avg(Sales) as 'Total Revenue', avg(Profit) as 'Total Profit', 
               avg(Profit)/avg(Sales)*100 as 'Profit Margin', 
               avg(Original_Price-Sales) as 'Total Discounts', 
               avg(Original_Price-Sales)/avg(Original_Price) as 'Discount Loss %', 
               avg(sales)/avg(original_price)*100 as 'Realization Rate',
               avg(Profit)/avg(original_price-sales) as 'ROD Profit',
               avg(Sales)/avg(original_price-sales) as 'ROD Revenue'
               FROM Superstore 
               """)
rows=cursor.fetchall()
column_names = [col[0] for col in cursor.description]
print('Grouped by Region, Discount Type')
print(tabulate(rows, headers=column_names, tablefmt='psql'))


Grouped by Region, Discount Type
+----------------+-----------------+----------------+-----------------+-------------------+-------------------+--------------------+--------------+---------------+
|   Total Volume |   Total Revenue |   Total Profit |   Profit Margin |   Total Discounts |   Discount Loss % |   Realization Rate |   ROD Profit |   ROD Revenue |
|----------------+-----------------+----------------+-----------------+-------------------+-------------------+--------------------+--------------+---------------|
|        3.78957 |         229.858 |        28.6569 |         12.4672 |            424.44 |          0.648695 |            35.1305 |    0.0675169 |      0.541556 |
+----------------+-----------------+----------------+-----------------+-------------------+-------------------+--------------------+--------------+---------------+


ROD is 'Return on Discount' and is a good metric to evaluate the effectiveness of discounts bringing in profit or revenue.
ROD Profit is how much profit results from discounts
ROD Revenue, similarly, is how much revenue is generated from discounts

With an average ROD Profit = 0.068, this reads as "For every $1 given away on discounts, we get $0.07 back in profit and $0.54 in revenue."
This is very low and suggests that discounts are too deep to generate meaingful profit.
Consider that average original price is $654, but average sales is $230.  After costs, average profit is $29.
The proportion of loss to discount for average sale is 65%.

Let's take a granular look at where the problems are...

In [106]:
#Groupby region

cursor.execute("""select Region, 
               avg(Quantity) as 'Total Volume',avg(Sales) as 'Total Revenue', avg(Profit) as 'Total Profit', 
               avg(Profit)/avg(Sales)*100 as 'Profit Margin', 
               avg(Original_Price-Sales) as 'Total Discounts', 
               avg(Original_Price-Sales)/avg(Original_Price) as 'Discount Loss %', 
               avg(sales)/avg(original_price)*100 as 'Realization Rate',
               avg(Profit)/avg(original_price-sales) as 'ROD Profit',
               avg(Sales)/avg(original_price-sales) as 'ROD Revenue'
               FROM Superstore 
               group by Region 
               """)
rows=cursor.fetchall()
column_names = [col[0] for col in cursor.description]
print('Grouped by Region, Discount Type')
print(tabulate(rows, headers=column_names, tablefmt='psql'))


Grouped by Region, Discount Type
+----------+----------------+-----------------+----------------+-----------------+-------------------+-------------------+--------------------+--------------+---------------+
| Region   |   Total Volume |   Total Revenue |   Total Profit |   Profit Margin |   Total Discounts |   Discount Loss % |   Realization Rate |   ROD Profit |   ROD Revenue |
|----------+----------------+-----------------+----------------+-----------------+-------------------+-------------------+--------------------+--------------+---------------|
| Central  |        3.7796  |         215.773 |        17.0927 |         7.92163 |           336.354 |          0.609197 |            39.0803 |    0.0508176 |      0.641504 |
| East     |        3.72823 |         238.336 |        32.1358 |        13.4834  |           429.857 |          0.643312 |            35.6688 |    0.0747593 |      0.554455 |
| South    |        3.83272 |         241.804 |        28.8577 |        11.9343  |          

- West:  Highest margins and highest original prices
- East:  Second best margins, higher sales and lower original price compared to West, normally would mean better profit, but higher incurred costs or budget items make profit lower
- South:  Highest avg sales price, lowest original price -- budget items, on par with East and West
- Central:  Very low margins, lower priced items with deep discounts.  Needs investigating

In [100]:
#Groupby region

cursor.execute("""select Region, Discount_Type, 
               sum(Quantity) as 'Total Volume',sum(Sales) as 'Total Revenue', sum(Profit) as 'Total Profit', 
               sum(Profit)/sum(Sales)*100 as 'Profit Margin', 
               sum(Original_Price-Sales) as 'Total Discounts', 
               sum(Original_Price-Sales)/sum(Original_Price) as 'Discount Loss %', 
               sum(sales)/sum(original_price)*100 as 'Realization Rate',
               sum(Profit)/sum(original_price-sales) as 'ROD Profit',
               sum(Sales)/sum(original_price-sales) as 'ROD Revenue'
               FROM Superstore 
               group by Region, Discount_Type 
               order by Region, case Discount_Type
                when 'None' then 0
                when 'Small' then 1
                when 'Medium' then 2
                when 'Large' then 3
                when 'Deep' then 4
                else 5 end""")
rows=cursor.fetchall()
column_names = [col[0] for col in cursor.description]
print('Grouped by Region, Discount Type')
print(tabulate(rows, headers=column_names, tablefmt='psql'))


Grouped by Region, Discount Type
+----------+-----------------+----------------+-----------------+----------------+-----------------+-------------------+-------------------+--------------------+--------------+---------------+
| Region   | discount_type   |   Total Volume |   Total Revenue |   Total Profit |   Profit Margin |   Total Discounts |   Discount Loss % |   Realization Rate |   ROD Profit |   ROD Revenue |
|----------+-----------------+----------------+-----------------+----------------+-----------------+-------------------+-------------------+--------------------+--------------+---------------|
| Central  | None            |           3151 |       243151    |       76125.4  |        31.3079  |       0           |          0        |           100      |              |               |
| Central  | Small           |           3168 |       128955    |       15973.2  |        12.3866  |  554496           |          0.811318 |            18.8682 |    0.0288067 |      0.232563 |
| 

Visual analysis in Power BI will help aid analysis to highlight patterns

In [99]:
cursor.execute("""select Region, sum(Quantity) as 'Total Volume',sum(Sales) as 'Total Revenue', sum(Profit) as 'Total Profit', 
               sum(Profit)/sum(Sales)*100 as 'Profit Margin', 
               sum(Original_Price-Sales) as 'Total Discounts', 
               sum(Original_Price-Sales)/sum(Original_Price) as 'Discount Loss %', 
               sum(sales)/sum(original_price)*100 as 'Realization Rate',
               sum(Profit)/sum(original_price-sales) as 'ROD Profit',
               sum(Sales)/sum(original_price-sales) as 'ROD Revenue'
               FROM Superstore 
               group by Region
               order by sum(Profit) desc 
               """)
rows=cursor.fetchall()
column_names = [col[0] for col in cursor.description]
print('Region Summary (Ordered by Profit Leaders)')
print(tabulate(rows, headers=column_names, tablefmt='psql'))

Region Summary (Ordered by Profit Leaders)
+----------+----------------+-----------------+----------------+-----------------+-------------------+-------------------+--------------------+--------------+---------------+
| Region   |   Total Volume |   Total Revenue |   Total Profit |   Profit Margin |   Total Discounts |   Discount Loss % |   Realization Rate |   ROD Profit |   ROD Revenue |
|----------+----------------+-----------------+----------------+-----------------+-------------------+-------------------+--------------------+--------------+---------------|
| West     |          12266 |          725458 |       108418   |        14.9448  |       1.74282e+06 |          0.706087 |            29.3913 |    0.0622088 |      0.416256 |
| East     |          10618 |          678781 |        91522.8 |        13.4834  |       1.22423e+06 |          0.643312 |            35.6688 |    0.0747593 |      0.554455 |
| South    |           6209 |          391722 |        46749.4 |        11.9343  |

Grouped by Product Category

In [ ]:
cursor.execute("""select Category, "Sub-Category", sum(Quantity) as 'Volume',sum(Sales) as 'Revenue', sum(Profit) as 'Profit', 
               sum(Profit)/sum(Sales)*100 as 'Profit_Margin',  
               sum(original_price-sales)/sum(original_price) as 'Discount Loss %', 
               sum(sales)/sum(original_price)*100 as 'Realization Rate'
               
               FROM Superstore 
               group by Category, "Sub-Category"
               order by "Profit" desc 
               """)
rows=cursor.fetchall()
column_names = [col[0] for col in cursor.description]
print('Region Summary (Ordered by Profit Leaders)')
print(tabulate(rows, headers=column_names, tablefmt='psql'))

Region Summary (Ordered by Profit Leaders)
+-----------------+----------------+----------+-----------+------------+-----------------+-------------------+--------------------+
| Category        | Sub-Category   |   Volume |   Revenue |     Profit |   Profit_Margin |   Discount Loss % |   Realization Rate |
|-----------------+----------------+----------+-----------+------------+-----------------+-------------------+--------------------|
| Technology      | Copiers        |      234 | 149528    |  55617.8   |        37.1956  |          0.625753 |            37.4247 |
| Technology      | Phones         |     3289 | 330007    |  44515.7   |        13.4893  |          0.691199 |            30.8801 |
| Technology      | Accessories    |     2976 | 167380    |  41936.6   |        25.0547  |          0.539431 |            46.0569 |
| Office Supplies | Paper          |     5178 |  78479.2  |  34053.6   |        43.3918  |          0.563433 |            43.6567 |
| Office Supplies | Binders      

In [109]:
cursor.execute("""select Category, "Discount_Type", sum(Quantity) as 'Total Volume',sum(Sales) as 'Total Revenue', sum(Profit) as 'Total Profit', 
               sum(Profit)/sum(Sales)*100 as 'Profit Margin', 
               sum(Original_Price-Sales) as 'Total Discounts', 
               sum(Original_Price-Sales)/sum(Original_Price) as 'Discount Loss %', 
               sum(sales)/sum(original_price)*100 as 'Realization Rate',
               sum(Profit)/sum(original_price-sales) as 'ROD Profit',
               sum(Sales)/sum(original_price-sales) as 'ROD Revenue',
               sum(original_price-sales)/sum(original_price) as 'Discount Loss %', 
               sum(sales)/sum(original_price)*100 as 'Realization Rate'
               FROM Superstore 
               group by Category, "Discount_Type"
               order by Category, case Discount_Type
                when 'None' then 0
                when 'Small' then 1
                when 'Medium' then 2
                when 'Large' then 3
                when 'Deep' then 4
                else 5 end 
               """)
rows=cursor.fetchall()
column_names = [col[0] for col in cursor.description]
print('Region Summary (Ordered by Profit Leaders)')
print(tabulate(rows, headers=column_names, tablefmt='psql'))

Region Summary (Ordered by Profit Leaders)
+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-------------------+-------------------+--------------------+--------------+---------------+-------------------+--------------------+
| Category        | discount_type   |   Total Volume |   Total Revenue |   Total Profit |   Profit Margin |   Total Discounts |   Discount Loss % |   Realization Rate |   ROD Profit |   ROD Revenue |   Discount Loss % |   Realization Rate |
|-----------------+-----------------+----------------+-----------------+----------------+-----------------+-------------------+-------------------+--------------------+--------------+---------------+-------------------+--------------------|
| Furniture       | None            |           3224 |       256025    |       58133.1  |        22.706   |       0           |          0        |           100      |              |               |          0        |           100 

Calculated profit margin and return on discount per product segment

I like my calculated columns I added in SQL.  Rather than go back and add them to the database through Pandas, I'd like to make them permanent and then export as a new .db file

In [ ]:
cursor.execute("""
    CREATE TABLE superstore_enriched AS
    SELECT *,
        CASE 
            WHEN Sales != 0 THEN Profit * 100.0 / Sales 
            ELSE 0 
        END AS Profit_Margin,

        (Original_Price - Sales) AS Discount_Loss,

        CASE 
            WHEN Original_Price != 0 THEN (Original_Price - Sales)*100 / Original_Price 
            ELSE 0 
        END AS Discount_Loss_Pct,

        CASE 
            WHEN Original_Price != 0 THEN Sales * 100.0 / Original_Price
            ELSE 100 
        END AS Realization_Rate,

        CASE 
            WHEN (Original_Price - Sales) != 0 THEN Profit * 1.0 / (Original_Price - Sales)
            ELSE NULL 
        END AS ROD_Profit,

        CASE 
            WHEN (Original_Price - Sales) != 0 THEN Sales * 1.0 / (Original_Price - Sales)
            ELSE NULL 
        END AS ROD_Revenue

    FROM Superstore
""")
#this saves the table to the existing 'superstore.db' file
connection.commit()


In [112]:
cursor.execute('select * from superstore_enriched limit 5')
rows=cursor.fetchall()
column_names = [col[0] for col in cursor.description]
print('Region Summary (Ordered by Profit Leaders)')
print(tabulate(rows, headers=column_names, tablefmt='psql'))

Region Summary (Ordered by Profit Leaders)
+----------------+--------------+-------------+----------------+---------------+-----------------+-----------+---------------+-----------------+------------+---------------+----------+-----------------+-----------------+----------------+-------------------------------------------------------------+---------+------------+------------+-----------+-----------------+------------------+-----------------+-----------------+---------------------+--------------------+--------------+---------------+
| Order ID       | Order Date   | Ship Date   | Ship Mode      | Customer ID   | Customer Name   | Segment   | Country       | City            | State      |   Postal Code | Region   | Product ID      | Category        | Sub-Category   | Product Name                                                |   Sales |   Quantity |   Discount |    Profit | discount_type   |   Original_Price |   Profit_Margin |   Discount_Loss |   Discount_Loss_Pct |   Realization_Rate 

## Now that the table with new features is created in database, lets visualize using Power BI!

In [113]:
# Close the SQLite connection
connection.close()